In [36]:
import sqlite3 as sql
import pandas  as pd
import numpy as np
#table_name = "dataset"
#read_data() for getting data from dataset
#get() for getting sequence,mutation,ddG for which we should make the prediction
#output() for sending result to server
#file_name = name_of_file.sqlite
def inverse_mutation(line):
    sequence,current,position,mutation,ddG,PH,Temp = line
    current,mutation = mutation,current
    sequence = sequence[:int(position)]+current+sequence[int(position)+1:]
    ddG = str(-float(ddG))
    return [sequence,current,position,mutation,ddG,PH,Temp]
def read_data(file_name):
    conn = sql.connect(file_name)
    cursor = conn.cursor()
    types = [str,str,int,str,float,float,float]
    names = ["sequence","current","position","mutation","ddG","PH","Temp"]
    results = {names[i]:[] for i in range(len(names))}
    for row in cursor.execute("SELECT * FROM dataset"):
        row = list(row)
        row[2] = str(int(row[2])-1)
        inversed = inverse_mutation(row)
        for i in range(len(names)):
            results[names[i]].append(types[i](row[i]))
            #results[names[i]].append(types[i](inversed[i]))
    result = pd.DataFrame(results)
    conn.close()
    return result
def get():
    text = raw_input()
    types = [str,str,int,str,float]
    split_text = text.split(" ")
    return [types[i](split_text[i]) for i in range(len(split_text))]
#values = [sequence,current,position,mutation,ddG]
def output(values):
    result = values[0]
    for i in range(1,len(values)):
        result += " "+ str(values[i])
    print (result)

In [63]:
def over_sampling(data_set):
    number_plus = len(data_set[data_set.ddG > 0])
    number_minus = len(data_set[data_set.ddG < 0])
    multiply = 0
    plus = 0
    new_number_plus = number_plus
    new_number_minus = number_minus
    while new_number_plus+number_plus < new_number_minus:
        new_number_plus += number_plus
        multiply += 1
    plus = new_number_minus - new_number_plus
    #need to do
    

In [64]:
raw_data = read_data("iwdb.sqlite")
raw_data.head()

,PH,Temp,current,ddG,mutation,position,sequence
0,7.4,25.0,L,-1.18,K,13,AAGHVNIAEAVQQLNHTIVNAAHELHETLGLPTPDEALNLLTEQAN...
1,8.5,20.0,L,0.81,S,36,MEPAAGSSMEPSADWLATAAARGRVEEVRALLEAGALPNAPNSYGR...
2,7.4,25.0,A,-1.40,K,136,AAGHVNIAEAVQQLNHTIVNAAHELHETLGLPTPDEALNLLTEQAN...
3,7.0,25.0,F,-3.70,A,11,AKLNESHRLHQFFRDMDDEESWIKEKKLLVSSEDYGRDLTGVQNLR...
4,7.0,25.0,F,-1.30,L,11,AKLNESHRLHQFFRDMDDEESWIKEKKLLVSSEDYGRDLTGVQNLR...


In [65]:
raw_data[['sequence','position']].head()

,sequence,position
0,AAGHVNIAEAVQQLNHTIVNAAHELHETLGLPTPDEALNLLTEQAN...,13
1,MEPAAGSSMEPSADWLATAAARGRVEEVRALLEAGALPNAPNSYGR...,36
2,AAGHVNIAEAVQQLNHTIVNAAHELHETLGLPTPDEALNLLTEQAN...,136
3,AKLNESHRLHQFFRDMDDEESWIKEKKLLVSSEDYGRDLTGVQNLR...,11
4,AKLNESHRLHQFFRDMDDEESWIKEKKLLVSSEDYGRDLTGVQNLR...,11


In [66]:
def neighbour(sequence,position,shift):
    if (position + shift > len(sequence)-1 or position + shift < 0):
        return "0"
    else:
        return sequence[position+shift]

In [67]:
number_neighbours = 3
for i in range(1,number_neighbours+1):
    raw_data['Left'+str(i)] = raw_data.apply(lambda row: neighbour(row['sequence'],row['position'],-(i)),axis=1)
    raw_data['Right'+str(i)] = raw_data.apply(lambda row: neighbour(row['sequence'],row['position'],(i)),axis=1)
raw_data.head()

,PH,Temp,current,ddG,mutation,position,sequence,Left1,Right1,Left2,Right2,Left3,Right3
0,7.4,25.0,L,-1.18,K,13,AAGHVNIAEAVQQLNHTIVNAAHELHETLGLPTPDEALNLLTEQAN...,Q,N,Q,H,V,T
1,8.5,20.0,L,0.81,S,36,MEPAAGSSMEPSADWLATAAARGRVEEVRALLEAGALPNAPNSYGR...,A,P,G,N,A,A
2,7.4,25.0,A,-1.40,K,136,AAGHVNIAEAVQQLNHTIVNAAHELHETLGLPTPDEALNLLTEQAN...,E,A,Q,E,L,K
3,7.0,25.0,F,-3.70,A,11,AKLNESHRLHQFFRDMDDEESWIKEKKLLVSSEDYGRDLTGVQNLR...,Q,F,H,R,L,D
4,7.0,25.0,F,-1.30,L,11,AKLNESHRLHQFFRDMDDEESWIKEKKLLVSSEDYGRDLTGVQNLR...,Q,F,H,R,L,D


In [68]:
raw_data = raw_data.sample(frac=1).reset_index(drop=True)
raw_data.head()

,PH,Temp,current,ddG,mutation,position,sequence,Left1,Right1,Left2,Right2,Left3,Right3
0,5.7,9.9,A,-2.30,G,19,GSQPAKKTYTWNTKEEAKQAFKELLKEKRVPSNASWEQAMKMIIND...,Q,F,K,K,A,E
1,6.2,25.0,D,-0.91,A,63,SSVEKKPEGVNTGAGDRHNLKTEWPELVGKSVEEAKKVILQDKPEA...,I,R,R,V,Y,R
2,7.5,25.0,I,-2.04,A,55,GVQVETISPGDGRTFPKRGQTCVVHYTGMLEDGKKFDSSRDRNKPF...,V,R,E,G,Q,W
3,9.0,50.0,C,-1.36,A,117,MERYESLFAQLKERKEGAFVPFVTLGDPGIEQSLKIIDTLIEAGAD...,Q,E,A,K,Y,V
4,7.0,25.0,I,-3.11,A,6,AECSVDIQGNDQMQFNTNAITVDKSCKQFTVNLSHPGNLPKNVMGH...,D,Q,V,G,S,N


In [69]:
len(raw_data)

2211

In [70]:
inputting = 'A B C D E F G H I J K L M N O P Q R S T U V W X Y Z 0'

In [71]:
symbols = inputting.split(' ')
numerics = [i for i in range(len(symbols))]

In [72]:
categorical = ['current','mutation']+['Left'+str(i) for i in range(1,number_neighbours+1)]+['Right'+str(i) for i in range(1,number_neighbours+1)]
for item in categorical:
    raw_data[item] = raw_data[item].apply(lambda x: numerics[symbols.index(x)])

In [73]:
raw_data.head()

,PH,Temp,current,ddG,mutation,position,sequence,Left1,Right1,Left2,Right2,Left3,Right3
0,5.7,9.9,0,-2.30,6,19,GSQPAKKTYTWNTKEEAKQAFKELLKEKRVPSNASWEQAMKMIIND...,16,5,10,10,0,4
1,6.2,25.0,3,-0.91,0,63,SSVEKKPEGVNTGAGDRHNLKTEWPELVGKSVEEAKKVILQDKPEA...,8,17,17,21,24,17
2,7.5,25.0,8,-2.04,0,55,GVQVETISPGDGRTFPKRGQTCVVHYTGMLEDGKKFDSSRDRNKPF...,21,17,4,6,16,22
3,9.0,50.0,2,-1.36,0,117,MERYESLFAQLKERKEGAFVPFVTLGDPGIEQSLKIIDTLIEAGAD...,16,4,0,10,24,21
4,7.0,25.0,8,-3.11,0,6,AECSVDIQGNDQMQFNTNAITVDKSCKQFTVNLSHPGNLPKNVMGH...,3,16,21,6,18,13


In [74]:
train_size = 0.8
size = int(len(raw_data)*train_size)
train_data = raw_data.iloc[:size]
test_data = raw_data.iloc[size:]
raw_data.shape,train_data.shape,test_data.shape

((2211, 13), (1768, 13), (443, 13))

In [75]:
train_labels = train_data['ddG'].values

In [76]:
train_data = train_data.drop(['ddG','sequence'],axis=1)

In [77]:
train_data.head()

,PH,Temp,current,mutation,position,Left1,Right1,Left2,Right2,Left3,Right3
0,5.7,9.9,0,6,19,16,5,10,10,0,4
1,6.2,25.0,3,0,63,8,17,17,21,24,17
2,7.5,25.0,8,0,55,21,17,4,6,16,22
3,9.0,50.0,2,0,117,16,4,0,10,24,21
4,7.0,25.0,8,0,6,3,16,21,6,18,13


In [78]:
train_labels

array([-2.3 , -0.91, -2.04, ...,  0.11, -0.6 , -3.68])

In [79]:
test_labels = test_data['ddG'].values

In [80]:
test_data = test_data.drop(['ddG','sequence'],axis=1)

In [81]:
test_data.head()

,PH,Temp,current,mutation,position,Left1,Right1,Left2,Right2,Left3,Right3
1768,7.0,20.0,12,0,31,15,19,16,5,6,17
1769,6.3,25.0,4,16,28,18,0,10,16,19,0
1770,7.4,20.0,12,8,20,19,18,21,2,17,10
1771,5.0,25.0,17,0,33,12,0,10,0,19,0
1772,7.0,20.0,16,0,130,0,0,4,10,18,10


In [125]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn import pipeline
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import learning_curve, GridSearchCV
from numpy import linalg as LA

In [87]:
binary_data_columns = ['holiday', 'workingday']
binary_data_indices = np.array([(column in binary_data_columns) for column in train_data.columns], dtype = bool)
categorical_data_indices = np.array([(column in categorical) for column in train_data.columns], dtype = bool)
numeric_data_columns = ['Temp', 'PH', 'position']
numeric_data_indices = np.array([(column in numeric_data_columns) for column in train_data.columns], dtype = bool)
categorical_data_indices

array([False, False,  True,  True, False,  True,  True,  True,  True,
        True,  True])

In [136]:
regr = RandomForestRegressor(random_state = 0, max_depth = 20, n_estimators = 50)
#regr = AdaBoostRegressor(random_state=0, n_estimators=100)
estimator = pipeline.Pipeline(steps = [       
    ('feature_processing', pipeline.FeatureUnion(transformer_list = [        
            #binary
            ('binary_variables_processing', preprocessing.FunctionTransformer(lambda data: data[:, binary_data_indices])), 
                    
            #numeric
            ('numeric_variables_processing', pipeline.Pipeline(steps = [
                ('selecting', preprocessing.FunctionTransformer(lambda data: data[:, numeric_data_indices]))
                        ])),
        
            #categorical
            ('categorical_variables_processing', pipeline.Pipeline(steps = [
                ('selecting', preprocessing.FunctionTransformer(lambda data: data[:, categorical_data_indices])),
                ('hot_encoding', preprocessing.OneHotEncoder(handle_unknown = 'ignore'))            
                        ])),
        ])),
    ('model_fitting', regr)
    ]
)

In [137]:
estimator.fit(train_data,train_labels)

Pipeline(memory=None,
     steps=[('feature_processing', FeatureUnion(n_jobs=None,
       transformer_list=[('binary_variables_processing', FunctionTransformer(accept_sparse=False, check_inverse=True,
          func=<function <lambda> at 0x000000000E7976D8>, inv_kw_args=None,
          inverse_func=None, kw_args=None, pass_y=...timators=50, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False))])

In [141]:
estimator.get_params().keys()

['feature_processing__binary_variables_processing__pass_y',
 'feature_processing__binary_variables_processing__func',
 'feature_processing__numeric_variables_processing__selecting__kw_args',
 'feature_processing__n_jobs',
 'feature_processing__numeric_variables_processing__selecting',
 'feature_processing__categorical_variables_processing__selecting__kw_args',
 'feature_processing__categorical_variables_processing__selecting__pass_y',
 'model_fitting__min_samples_split',
 'feature_processing__transformer_weights',
 'model_fitting__min_samples_leaf',
 'feature_processing__categorical_variables_processing__hot_encoding__n_values',
 'model_fitting__n_estimators',
 'model_fitting__min_impurity_split',
 'feature_processing__categorical_variables_processing__selecting__func',
 'model_fitting__criterion',
 'feature_processing__categorical_variables_processing__selecting__check_inverse',
 'feature_processing__binary_variables_processing__check_inverse',
 'feature_processing',
 'feature_process

In [142]:
parameters_grid = {
    'model_fitting__n_estimators' = [i for i in range(100,5001)],
    'model_fitting__max_depth' = [j for j in range(10,201)]
}

SyntaxError: invalid syntax (<ipython-input-142-1b0940cb9f0c>, line 2)

In [143]:
grid_cv = GridSearchCV(estimator, parameters_grid, cv = 4)

In [144]:
grid_cv.fit(train_data, train_labels)

ValueError: Invalid parameter n_estimators for estimator Pipeline(memory=None,
     steps=[('feature_processing', FeatureUnion(n_jobs=None,
       transformer_list=[('binary_variables_processing', FunctionTransformer(accept_sparse=False, check_inverse=True,
          func=<function <lambda> at 0x000000000E7976D8>, inv_kw_args=None,
          inverse_func=None, kw_args=None, pass_y=...timators=50, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False))]). Check the list of available parameters with `estimator.get_params().keys()`.

In [145]:
print grid_cv.best_score_
print grid_cv.best_params_

AttributeError: 'GridSearchCV' object has no attribute 'best_score_'

In [138]:
metrics.mean_absolute_error(test_labels, estimator.predict(test_data))

0.8974552010023411

In [139]:
def test_true_false(label, true_label):
    count_true = 0
    count_all = 0
    for i in range(len(label)):
        count_all += 1
        if (np.sign(true_label[i]) == np.sign(label[i])):
            count_true += 1
    return float(count_true)/count_all
test_true_false(estimator.predict(test_data),test_labels)

0.835214446952596

In [134]:
def test_symmetry(test_data,test_labels):
    columns = map(str,test_data.columns)
    inversed_test_data = test_data.copy()
    inversed_test_data.current,inversed_test_data.mutation = inversed_test_data.mutation.apply(lambda x: x),inversed_test_data.current.apply(lambda x: x)
    inversed_labels = estimator.predict(inversed_test_data)
    test_labels = np.array(test_labels)
    inversed_labels = np.array(inversed_labels)
    vector = test_labels+inversed_labels
    return LA.norm(vector)/len(vector)

In [140]:
test_symmetry(test_data,test_labels)

0.13553754956883332

In [124]:
test_data.head()

,PH,Temp,current,mutation,position,Left1,Right1,Left2,Right2,Left3,Right3
1768,7.0,20.0,12,0,31,15,19,16,5,6,17
1769,6.3,25.0,4,16,28,18,0,10,16,19,0
1770,7.4,20.0,12,8,20,19,18,21,2,17,10
1771,5.0,25.0,17,0,33,12,0,10,0,19,0
1772,7.0,20.0,16,0,130,0,0,4,10,18,10
